In [ ]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.7/dist-packages (2.4)


In [ ]:
!unzip drive/MyDrive/f.zip

Archive:  drive/MyDrive/f.zip
replace 17_G_1623593016.2441733.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace 17_G_1623593016.2441733.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: 17_G_1623593016.2441733.pt  
  inflating: 11_G_1623586219.6545973.pt  
  inflating: 13_G_1623588456.3293748.pt  
  inflating: 18_G_1623594149.417595.pt  
  inflating: 14_G_1623589595.1418877.pt  
  inflating: 15_G_1623590736.230504.pt  
  inflating: 19_G_1623595289.1398213.pt  
  inflating: 10_G_1623585157.7964432.pt  
  inflating: 9_G_1623584122.1996632.pt  
  inflating: 12_G_1623587323.4433503.pt  
  inflating: 16_G_1623591874.1691759.pt  


In [ ]:
!unzip drive/MyDrive/models.zip

Archive:  drive/MyDrive/models.zip
replace models/19_D_1620024060.7773879.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: models/19_D_1620024060.7773879.pt  
  inflating: models/19_G_1620024060.8170185.pt  
  inflating: models/18_D_1620023445.871576.pt  
  inflating: models/17_D_1620022844.0652235.pt  
  inflating: models/18_G_1620023445.9103527.pt  
  inflating: models/17_G_1620022844.1141694.pt  
  inflating: models/16_D_1620022253.2658226.pt  
  inflating: models/15_D_1620021671.2213771.pt  
  inflating: models/14_D_1620021100.2137222.pt  
  inflating: models/13_D_1620020532.8641236.pt  
  inflating: models/15_G_1620021671.2555697.pt  
  inflating: models/13_G_1620020532.8982918.pt  
  inflating: models/12_G_1620019965.689112.pt  
  inflating: models/11_D_1620019397.9907248.pt  
  inflating: models/10_D_1620018831.0869498.pt  
  inflating: models/11_G_1620019398.0249395.pt  
  inflating: models/9_D_1620018264.0648942.pt  
  inflating: models/8_D_1620017697.3197768.pt  
  in

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from skimage.color import lab2rgb

import torch
from torchvision import transforms
from torchvision.models.resnet import resnet18

from fastai.vision.learner import create_body
from fastai.vision.models.unet import DynamicUnet

In [ ]:
SIZE = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_generator(n_inputs, n_outputs, core):
  body = create_body(core, pretrained=True, n_in=n_inputs, cut=-2)

  return DynamicUnet(body, n_outputs, (SIZE, SIZE))

def lab_to_rgb(L, ab): 
    L = (L + 1.) * 50.
    ab = ab * 110.
    Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()
    rgb_imgs = []
    for img in Lab:
        img_rgb = lab2rgb(img)
        rgb_imgs.append(img_rgb)
    return np.stack(rgb_imgs, axis=0)

In [ ]:
G1 = build_generator(1, 2, resnet18).to(device)

G1.load_state_dict(torch.load("14_G_1623589595.1418877.pt", map_location=device))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


<All keys matched successfully>

In [ ]:
G2 = build_generator(1, 2, resnet18).to(device)

G2.load_state_dict(torch.load("models/14_G_1620021100.248362.pt", map_location=device))

<All keys matched successfully>

In [ ]:
inputs = [Image.open(f"./{i}.png") for i in range(1, 5)]

tensors = torch.empty(0, 1, SIZE, SIZE).to(device)

for img in inputs:
  t = (2 * (transforms.ToTensor()(img))[0] - 1).unsqueeze(0).unsqueeze(0).to(device)
  tensors = torch.cat((tensors, t), dim=0)

print(tensors.shape)

torch.Size([4, 1, 256, 256])


In [ ]:
out1 = G1(tensors)

print(out1.shape)

torch.Size([4, 2, 256, 256])


In [ ]:
out2 = G2(tensors)

print(out2.shape)

torch.Size([4, 2, 256, 256])


In [ ]:
from skimage.color import rgb2lab
from torchvision import transforms

labels = [Image.open(f"./{i}.jpg") for i in range(1, 5)]

labels = [transforms.ToTensor()(rgb2lab(np.array(i)).astype("float32")).to(device) for i in labels]

print(labels[0].shape)

torch.Size([3, 256, 256])


In [ ]:
l1 = [0, 0, 0, 0]

for l in range(4):
    for k in range(2):
        for i in range(256):
            for j in range(256):
                l1[l] += (110*out1[l, k, i, j] - labels[l][k + 1, i, j])**2
                                                     
l1 = [l/256**2 for l in l1]
print(l1)

[TensorBase(481.9427, device='cuda:0', grad_fn=<AliasBackward>), TensorBase(625.9880, device='cuda:0', grad_fn=<AliasBackward>), TensorBase(748.5225, device='cuda:0', grad_fn=<AliasBackward>), TensorBase(1256.3251, device='cuda:0', grad_fn=<AliasBackward>)]


In [ ]:
l2 = [0, 0, 0, 0]

for l in range(4):
    for k in range(2):
        for i in range(256):
            for j in range(256):
                l2[l] += (110*out2[l, k, i, j] - labels[l][k + 1, i, j])**2
                                                     
l2 = [l/256**2 for l in l2]
print(l2)

[TensorBase(301.3324, device='cuda:0', grad_fn=<AliasBackward>), TensorBase(405.6307, device='cuda:0', grad_fn=<AliasBackward>), TensorBase(818.9330, device='cuda:0', grad_fn=<AliasBackward>), TensorBase(884.4907, device='cuda:0', grad_fn=<AliasBackward>)]
